In [28]:
from bs4 import BeautifulSoup
import requests
import re
import json

In [29]:
def get_html(url):
    response = requests.get(url)
    return response.text

In [3]:
tvs_url = 'https://rozetka.com.ua/all-tv/c80037/'

In [55]:
from collections import namedtuple

CatalogPage = namedtuple('CatalogPage', 'max_page_num items')

def get_items(catalog_url, page_num):
    page_url = catalog_url if page_num == 0 else catalog_url + 'page={}/'.format(page_num + 1)
    
    soup = BeautifulSoup(get_html(tvs_url), 'lxml')
    
    title_divs = tv_soup.find_all('div', class_ = 'g-i-tile-i-title clearfix') 

    titles = []
    for div in title_divs:
        a = div.find('a')
        titles.append((a.get("href"), a.text.strip()))
        
    page_items = tv_soup.find_all('li', class_ = 'paginator-catalog-l-i pos-fix')

    max_page_num = [int(item.get('id').replace('page', '').strip()) for item in page_items][-1]    
    
    return CatalogPage(max_page_num, titles)

In [26]:
page_0 = get_items(tvs_url, 0)

In [27]:
len(page_1.items)

32

In [24]:
page_1 = get_items(tvs_url, 1)

In [25]:
len(page_1.items)

32

In [32]:
item_url = 'https://rozetka.com.ua/philips_43pus6503_12/p39097504/'

In [68]:
Comment = namedtuple('Comment', 'text up_count down_count replies')


CommentsPage = namedtuple('CommentsPage', 'max_page_num comments')

def get_comments(item_url, page_num):
    comments_url = item_url + 'comments/'
    page_url = comments_url if page_num == 0 else comments_url + 'page={}/'.format(page_num + 1)
    
    soup = BeautifulSoup(get_html(page_url), 'lxml')
    article_divs = soup.find_all('article', class_ = 'pp-review-i')
    
    comments = []
    
    for div in article_divs:
        text = [text_div.text.strip().replace(u'\xa0', ' ') \
                for text_div in div.find_all('div', class_ = 'pp-review-text-i')]
    
        replies = [rep_div.text.strip().replace(u'\xa0', ' ') \
                   for rep_div in div.find_all('div', class_ = 'pp-replies-text')]
    
        up_count = div.find('a', class_ = 'pp-review-vote-positive').\
            find('span', class_ = 'pp-review-vote-count').text.strip()
        
        down_count = div.find('a', class_ = 'pp-review-vote-negative').\
            find('span', class_ = 'pp-review-vote-count').text.strip()
    
        comments.append(Comment(text, int(up_count) if up_count else 0, \
                                int(down_count) if down_count else 0, replies))
    
    page_items = soup.find_all('li', class_ = 'paginator-catalog-l-i')
    
    max_page_num = [int(item.get('id').replace('page', '').strip()) for item in page_items][-1]
    
    return CommentsPage(max_page_num, comments)

In [69]:
get_comments(item_url, 0)

CommentsPage(max_page_num=10, comments=[Comment(text=['До какого числа будет идти акция на данную модель?'], up_count=0, down_count=0, replies=[]), Comment(text=['', 'Недостатки: Постоянно сбрасываются настройки, перестали работать настройки сети'], up_count=0, down_count=1, replies=[]), Comment(text=['Подскажите пожалуйста для Ps4 Pro подойдет не будет ли флагов??'], up_count=0, down_count=2, replies=[]), Comment(text=['Добрый вечер! Кто может подсказать как подключить акустику через блютуз на данной модели ??? Нигде не могу найти. Буду благодарен за помощь!'], up_count=1, down_count=1, replies=[]), Comment(text=['Розетка не дала гарантийный талон потом прислали но при обращении в сервисный центр сказали что эта партия в Украине не зарегистрирована', 'Достоинства: Хорошо работал', 'Недостатки: Но не долго ремонт очень дорого по гарантии не чинят'], up_count=1, down_count=1, replies=['Это кто сказал такое? В сервис самой Розетки обращались?']), Comment(text=['У мен перестали запускатьс

In [70]:
get_comments(item_url, 1)

CommentsPage(max_page_num=10, comments=[Comment(text=['Добрий день. Чи встановлюється на цей телевізор Oll.TV і Megogo?'], up_count=0, down_count=1, replies=['Добрий день!\nДані сервіси передвстановлені у всіх смарт-телевізорах Philips.']), Comment(text=['А какая оперативка и жёсткий?'], up_count=2, down_count=2, replies=[]), Comment(text=['Поддерживает ли этот тв голосовое управление (поиск)? Если да, то какой пульт подойдёт?'], up_count=0, down_count=1, replies=['Здравствуйте, Максим! Данная модель телевизора на имеет функции голосового поиска.']), Comment(text=['Софи сырая плотформа глючит постоянно', 'Достоинства: Екран', 'Недостатки: Софи'], up_count=0, down_count=2, replies=['Евгений, здравствуйте!\nВозможно вы поделитесь информацией о ваших проблемах с данным ТВ?', 'Заказал две недели назад. Сейчас уже вернул ьы и поменял на Самсунг. Странная картинка в динамике при просмотре того же футбола на Megogo или OLLtv. Еще очень глючит в этих приложениях, постоянно что-то не так происх